分类回归树
====
CART 算法，英文全称叫做 Classification And Regression Tree，中文叫做分类回归树。ID3 和 C4.5 算法可以生成二叉树或多叉树，而 CART 只支持二叉树。同时 CART 决策树比较特殊，既可以作**分类树，又可以作回归树**。

### 分類樹
* 如果我构造了一棵决策树，想要基于**数据判断**这个人的职业身份，这个就属于**分类树**，因为是从几个分类中来做选择。
* 处理**离散数据**，也就是数据种类**有限的数据**，它输出的是**样本的类别**

### 回歸樹
* 如果是给定了数据，想要**预测**这个人的年龄，那就属于**回归树**。
* 对连续型的数值进行预测，也就是数据在某个区间内都有取值的可能，它输出的是一个**数值**。

## 1. CART 分类树的工作流程
CART 分类树与 C4.5 算法类似，只是属性选择的指标采用的是基尼系数。基尼係數用来衡量一个国家收入差距的常用指标。当基尼系数大于 0.4 的时候，说明财富差异悬殊。基尼系数在 0.2-0.4 之间说明分配合理，财富差距不大。

基尼系数越小的时候，说明样本之间的差异性小，不确定程度低。分类的过程本身是一个**不确定度降低的过程**，即纯度的提升过程。所以 CART 算法在构造分类树的时候，会选择**基尼系数最小的属性**作为属性的划分。
<img src="./images/18-02.png">

**p(Ck|t) 表示节点 t 属于类别 Ck 的概率，节点 t 的基尼系数为 1 减去各类别 Ck 概率平方和。**

* C為Choice的簡寫，(選擇該選項的個數)/(總筆數) = Ck/t，例如：該集合中總共6筆資料，其中選擇打球、不打球的人數分別為4與2，則C1=4, C2=2, t為6。

通过下面这个例子，我们计算一下两个集合的基尼系数分别为多少：

||集合|Gini|樣本穩定性|
|----|:----|:----|:----:|
|1|6 个都去打篮球|所有人都去打篮球，所以 p(Ck\|t)=1，因此 GINI(t)=1-1=0。|較小，穩定|
|2|3 个去打篮球，3 个不去打篮球|有一半人去打篮球，而另一半不去打篮球，所以，p(C1\|t)=0.5，p(C2\|t)=0.5，GINI(t)=1-（0.5\*0.5+0.5\*0.5）=0.5|較大，不穩定性更大|

在 CART 算法中，基于基尼系数对**特征属性**进行**二元分裂**，假设属性 A 将节点 D 划分成了 D1 和 D2，如下图所示：
<img src="./images/18-03.jpg">

节点 D 的基尼系数等于子节点 D1 和 D2 的**归一化基尼系数之和**，用公式表示为：
<img src="./images/18-04.png">

归一化基尼系数代表的是每个子节点的基尼系数乘以该节点占整体父亲节点 D 中的比例。上面我们已经计算了集合 D1 和集合 D2 的 GINI 系数，得到：
* GINI(D1) = 0
* GINI(D2) = 0.5

所以在属性 A 的划分下，节点 D 的基尼系数为：
* GINI(D,A) = 6/12\*GINI(D1) + 6/12\*GINI(D2) = 0.25

节点 D 被属性 A 划分后的**基尼系数越大**，样本集合的**不确定性越大**，也就是**不纯度越高**。


### 如何使用 CART 算法来创建分类树
1. CART 分类树实际上是基于基尼系数来做**属性划分**的。
2. 在 Python 的 sklearn 中，如果我们想要创建 CART 分类树，可以直接使用 **DecisionTreeClassifier** 这个类。创建这个类的时候，默认情况下 criterion 这个参数等于 gini，也就是按照基尼系数来选择属性划分，即默认采用的是 CART 分类树。

下面，我们来用 CART 分类树，给 iris 数据集构造一棵分类决策树。在 sklearn 中也自带了这个数据集。基于 iris 数据集，构造 CART 分类树的代码如下：
```python

# encoding=utf-8
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
# 准备数据集
iris=load_iris()
# 获取特征集和分类标识
features = iris.data
labels = iris.target
# 随机抽取33%的数据作为测试集，其余为训练集
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.33, random_state=0)
# 创建CART分类树
clf = DecisionTreeClassifier(criterion='gini')
# 拟合构造CART分类树
clf = clf.fit(train_features, train_labels)
# 用CART分类树做预测
test_predict = clf.predict(test_features)
# 预测结果与测试集结果作比对
score = accuracy_score(test_labels, test_predict)
print("CART分类树准确率 %.4lf" % score)
```
運作結果：
```python
CART分类树准确率 0.9600
```

如果我们把决策树画出来，可以得到下面的图示：
<img src="./images/18-05.png">

* 首先 train_test_split 可以帮助我们把数据集抽取一部分作为测试集，这样我们就可以得到训练集和测试集。
* 使用 clf = DecisionTreeClassifier(criterion=‘gini’) 初始化一棵 CART 分类树。这样你就可以对 CART 分类树进行训练。
* 使用 clf.fit(train_features, train_labels) 函数，将训练集的特征值和分类标识作为参数进行拟合，得到 CART 分类树。
* 使用 clf.predict(test_features) 函数进行预测，传入测试集的特征值，可以得到测试结果 test_predict。
* 最后使用 accuracy_score(test_labels, test_predict) 函数，传入测试集的预测结果与实际的结果作为参数，得到准确率 score。
* 我们能看到 sklearn 帮我们做了 CART 分类树的使用封装，使用起来还是很方便的。

## 2. CART 回歸樹的工作流程
### 離算程度，樣本x與均值μ之差的絕對值。
**分類樹**與**回歸樹**的過程一樣，但**回歸樹**得到的預測結果為**連續的**，而且判斷**不純度**的指標不同。分類樹用**基尼係數**判斷不純度，回歸樹用**離散程度**即資料的混亂程度判斷不純度。
樣本x減去樣本平均數，其值為**最小絕對偏差(LAD)**: 
<img src="./images/18-06.png">
**變異數(標準差的平方)** 公式，較常用**最小二乘偏差**:
<img src="./images/18-07.png">

### 如何使用CART回歸樹做預測
使用Sklearn的**波士頓房價數據集**，包含影響房價的指標有**犯罪率、房產稅等**，結果為房價。
```python
# encoding=utf-8
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.tree import DecisionTreeRegressor
# 准备数据集
boston=load_boston()
# 探索数据
print(boston.feature_names)
# 获取特征集和房价
features = boston.data
prices = boston.target
# 随机抽取33%的数据作为测试集，其余为训练集
train_features, test_features, train_price, test_price = train_test_split(features, prices, test_size=0.33)
# 创建CART回归树
dtr=DecisionTreeRegressor()
# 拟合构造CART回归树
dtr.fit(train_features, train_price)
# 预测测试集中的房价
predict_price = dtr.predict(test_features)
# 测试集的结果评价
print('回归树二乘偏差均值:', mean_squared_error(test_price, predict_price))
print('回归树绝对值偏差均值:', mean_absolute_error(test_price, predict_price)) 
```
運行結果: 
```python
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO' 'B' 'LSTAT']
回归树二乘偏差均值: 23.80784431137724
回归树绝对值偏差均值: 3.040119760479042
```
回歸樹畫出來如下圖示，因數據集指標有些多，所以樹比較大:
<img src="./images/18-08.png">

1. 透過tran_test_split將數據集分為**測試集、訓練集**。
2. 使用dtr = DecisionTreeRegression()**初始化**一棵CART回歸樹
3. 使用dtr.fit(train_features, train_price)函數，將**訓練集的特徵值**和**結果**作為參數進行擬合，得到回歸樹。
4. 使用dtr.predict(test_features)函數進行預測，傳入**測試集的特徵值**，可以得到預測結果predict_price。
5. 用mean_squared_error()求二乘偏差均值(變異數)、mean_abdolute_error()求絕對值偏差均值。

## 3. 決策樹的剪枝
CCP方法，為**後剪枝**方法，全稱cost-complexity prun**代價複雜度**。使用指標叫做**節點的表面誤差率增益值**。以作為**剪枝後誤差**的定義: 
<img src="./images/18-09.png">
1. Tt為以t為根節點的子樹，C(Tt)表示節點t的子樹**沒被裁剪時Tt的誤差**，C(t)表示節點t的子樹被**剪枝後節點t的誤差**，|Tt|代子樹Tt的葉子數，剪枝後，T的葉子數減少了|Tt|-1。
2. 「節點的表面誤差率增益值」等於節點t的子樹被剪枝後的誤差變化除以剪掉的葉子數量。
3. 希望剪枝後誤差最小，故尋找最小小α(alpha)值對應的節點，把它剪掉。這時生成第一個子樹，重複上面的過程，繼續剪枝，直到最後只剩下根節點，即最後一個子樹。
4. 得到剪枝後子樹集合後，需要驗證最所有子樹的誤差計算一遍。可以通過計算每個子樹的誤差計算一遍。可以通過計算每個子樹的**基尼指數**或者平方誤差，取**誤差最小**的那個樹，得到我們想要的結果。

## 總結
今天我給你講了CART決策樹，它是一棵決策二叉樹，既可以做分類樹，也可以做回歸樹。
1. 分類樹，CART採用基尼係數作為分段劃分的依據，得到的是離散的結果，也就是分類結果
2. 回歸樹，CART可以採用最小絕對偏差（LAD），或者最小二乘偏差（LSD）作為劃分的依據，得到的是連續值，即回歸預測結果。

三種決策樹之間的屬性選擇標准上的差異：
* ID3算法，基於信息增益做判斷； 
* C4.5算法，基於信息增益率做判斷； 
* CART算法，分類樹是基於基尼係數實際上，這三個指標也是計算“不純度”的三種計算方式。在工具使用上，我們可以使用sklearn中的DecisionTreeClassifier創建CART分類樹，通過DecisionTreeRegressor創建CART回歸樹。你可以用代碼自己跑一遍我在演示文稿中舉到的例子。
<img src="./images/18-10.png">


思考題
====
1. 你能說下ID3，C4.5，以及CART分類樹在做分區劃分時的區別嗎？
2. 第二個問題是，sklearn中有個手寫數字數據集，調用的 方法是load_digits（），您能否創建一個CART分類樹，對手寫數字數據集做分類？另外選擇一部分測試集，統計下分類樹的準確率？

In [1]:
# encoding=utf-8
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
# 准备数据集
iris=load_iris()
# 获取特征集和分类标识
features = iris.data
labels = iris.target
# 随机抽取33%的数据作为测试集，其余为训练集，返回三個array，訓練特徵值、測試特徵值、訓練標籤
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.33, random_state=0)
# 创建CART分类树
clf = DecisionTreeClassifier(criterion='gini')

# 拟合构造CART分类树
clf = clf.fit(train_features, train_labels)

# 用CART分类树做预测
test_predict = clf.predict(test_features)

# 预测结果与测试集结果作比对
score = accuracy_score(test_labels, test_predict)
print("CART分类树准确率 %.4lf" % score)

CART分类树准确率 0.9600


In [5]:
help(iris)

Help on Bunch in module sklearn.utils object:

class Bunch(builtins.dict)
 |  Bunch(**kwargs)
 |  
 |  Container object exposing keys as attributes.
 |  
 |  Bunch objects are sometimes used as an output for functions and methods.
 |  They extend dictionaries by enabling values to be accessed by key,
 |  `bunch["value_key"]`, or by an attribute, `bunch.value_key`.
 |  
 |  Examples
 |  --------
 |  >>> b = Bunch(a=1, b=2)
 |  >>> b['b']
 |  2
 |  >>> b.b
 |  2
 |  >>> b.a = 3
 |  >>> b['a']
 |  3
 |  >>> b.c = 6
 |  >>> b['c']
 |  6
 |  
 |  Method resolution order:
 |      Bunch
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __dir__(self)
 |      Default dir() implementation.
 |  
 |  __getattr__(self, key)
 |  
 |  __init__(self, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __setattr__(self, key, value)
 |      Implement setattr(self, name, value).
 |  
 |  __setstate__(self, state)
 |  
 |  ---------

In [8]:
# encoding=utf-8
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.tree import DecisionTreeRegressor
# 准备数据集
boston=load_boston()
# 探索数据
print(boston.feature_names)
# 获取特征集和房价
features = boston.data
prices = boston.target
# 随机抽取33%的数据作为测试集，其余为训练集
train_features, test_features, train_price, test_price = train_test_split(features, prices, test_size=0.33)
# 创建CART回归树
dtr=DecisionTreeRegressor()
# 拟合构造CART回归树
dtr.fit(train_features, train_price)
# 预测测试集中的房价
predict_price = dtr.predict(test_features)
# 测试集的结果评价
print('回归树二乘偏差均值:', mean_squared_error(test_price, predict_price))
print('回归树绝对值偏差均值:', mean_absolute_error(test_price, predict_price)) 

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
回归树二乘偏差均值: 189.97898203592814
回归树绝对值偏差均值: 10.373053892215568


In [14]:
boston.data[0]
boston.target[0]

24.0